<a href="https://colab.research.google.com/github/jamesluttringer2019/DS-Unit-2-Kaggle-Challenge/blob/master/module3/LS_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [0]:
train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [0]:
train, val = train_test_split(train, train_size=.8, test_size=.2,
                              stratify=train['status_group'], random_state=13)

In [0]:
#create a list of features to encode ordinally
ordinal_cols = ['lga', 'scheme_name', 'funder', 'installer']

In [0]:
import category_encoders as ce
import numpy as np
from scipy.stats import randint
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

In [0]:
def wrangle(X):
  '''Used to quickly wrangle sets from the data in the same way'''

  #prevent warning
  X = X.copy()

  #convert date to datetime
  X['date_recorded']=pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
  #Treat near zero values as zero in latitude
  X['latitude'] = X['latitude'].replace(-2e-08, 0)

  #Replace zeros with null values where appropriate
  zero_cols = ['latitude', 'longitude', 'construction_year', 'gps_height',]
  for col in zero_cols:
    X[col] = X[col].replace(0, np.nan)
  
  #drop duplicates, irrelevant, high cardinality, or unusable columns
  cols_to_drop = ['ward', 'num_private', 'recorded_by',
                  'subvillage', 'wpt_name',
                  'extraction_type', 'extraction_type_class',
                  'management','payment_type', 'waterpoint_type_group',
                  'source_type', 'water_quality', 'quantity_group',
                  'district_code','region_code']
  X = X.drop(columns = cols_to_drop)

  #create column to calculate years between construction and inspection
  X['constr_to_inspect'] = (X['date_recorded'].dt.year) - X['construction_year']
  
  #remove the colunms we used to create 'constr_to_inspect'
  #they are no longer needed
  X = X.drop(columns=['date_recorded', 'construction_year'])
  #return wrangled df
  return X

In [0]:
train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [0]:
target = 'status_group'
features = train.columns.drop(['status_group','id'])

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [0]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(cols=ordinal_cols),
    ce.OneHotEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=13)
)

In [0]:
param_distributions = {
    'simpleimputer__strategy':['mean','most_frequent','median'],
    'randomforestclassifier__n_estimators': randint(50,200),
    'randomforestclassifier__min_samples_leaf': randint(1,4)
}

In [0]:
search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter= 30,
    cv=5,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    random_state=13,
    n_jobs=-1
)

In [0]:
search.fit(X_train,y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   25.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   52.7s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  8.8min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=['lga',
                                                                   'scheme_name',
                                                                   'funder',
                                                                   'installer'],
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('one

In [0]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,10,18,5,2,20,23,13,27,29,6,12,14,17,11,1,3,21,0,25,15,8,7,9,28,24,22,19,16,4,26
mean_fit_time,29.5167,33.6187,34.8752,30.4773,17.3475,26.3437,23.6205,17.6584,19.1863,22.0217,14.7236,28.9761,26.8045,22.7203,23.2892,22.3718,22.9169,19.9814,18.6709,22.5989,25.6296,13.6954,14.709,12.3872,15.5221,34.8715,28.247,35.2677,12.7946,15.9919
std_fit_time,0.430132,0.286137,0.608024,1.3142,0.502389,0.796926,0.795297,0.404242,0.427205,0.93235,0.670905,0.606893,1.18342,1.33947,0.49613,1.26374,0.590514,0.510208,1.07898,0.533793,0.601344,0.591423,0.208628,0.417427,0.441706,0.942026,1.21877,0.746362,0.507987,0.579602
mean_score_time,1.29323,1.31883,1.25765,1.3922,0.816454,1.09815,0.908035,0.826987,0.737046,0.918981,0.699598,1.13043,1.0774,1.06237,1.05155,0.981696,0.870679,0.846538,0.82949,0.988851,0.991397,0.740606,0.788003,0.635999,0.733792,1.32203,1.07188,1.38906,0.756512,0.811209
std_score_time,0.204276,0.131001,0.086364,0.133555,0.0464762,0.0807413,0.0730089,0.0049465,0.0155125,0.0734701,0.0340184,0.12245,0.0739289,0.16548,0.0560661,0.085212,0.0393899,0.0222043,0.0541525,0.0651034,0.0451607,0.0331988,0.0838751,0.0990696,0.0211726,0.0963887,0.0224953,0.205078,0.0333998,0.0581175
param_randomforestclassifier__min_samples_leaf,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,1,3,3,1,1,1,1,1,1,1
param_randomforestclassifier__n_estimators,183,197,197,176,91,163,121,100,133,125,74,173,159,138,148,124,131,124,103,136,126,74,84,54,76,191,135,188,52,72
param_simpleimputer__strategy,mean,most_frequent,median,median,mean,most_frequent,most_frequent,most_frequent,most_frequent,most_frequent,most_frequent,median,median,most_frequent,mean,most_frequent,mean,mean,most_frequent,median,median,mean,mean,median,mean,most_frequent,most_frequent,mean,mean,most_frequent
params,{'randomforestclassifier__min_samples_leaf': 2...,{'randomforestclassifier__min_samples_leaf': 2...,{'randomforestclassifier__min_samples_leaf': 2...,{'randomforestclassifier__min_samples_leaf': 2...,{'randomforestclassifier__min_samples_leaf': 2...,{'randomforestclassifier__min_samples_leaf': 2...,{'randomforestclassifier__min_samples_leaf': 2...,{'randomforestclassifier__min_samples_leaf': 2...,{'randomforestclassifier__min_samples_leaf': 2...,{'randomforestclassifier__min_samples_leaf': 2...,{'randomforestclassifier__min_samples_leaf': 2...,{'randomforestclassifier__min_samples_leaf': 3...,{'randomforestclassifier__min_samples_leaf': 3...,{'randomforestclassifier__min_samples_leaf': 3...,{'randomforestclassifier__min_samples_leaf': 3...,{'randomforestclassifier__min_samples_leaf': 3...,{'randomforestclassifier__min_samples_leaf': 3...,{'randomforestclassifier__min_samples_leaf': 3...,{'randomforestclassifier__min_samples_leaf': 3...,{'randomforestclassifier__min_samples_leaf': 3...,{'randomforestclassifier__min_samples_leaf': 1...,{'randomforestclassifier__min_samples_leaf': 3...,{'randomforestclassifier__min_samples_leaf': 3...,{'randomforestclassifier__min_samples_leaf': 1...,{'randomforestclassifier__min_samples_leaf': 1...,{'randomforestclassifier__min_samples_leaf': 1...,{'randomforestclassifier__min_samples_leaf': 1...,{'randomforestclassifier__min_samples_leaf': 1...,{'randomforestclassifier__min_samples_leaf': 1...,{'randomforestclassifier__min_samples_leaf': 1...
split0_test_score,0.806187,0.804082,0.804714,0.804924,0.804819,0.804398,0.803241,0.802609,0.803241,0.803451,0.802504,0.800505,0.801031,0.801978,0.802399,0.802189,0.801242,0.801662,0.801978,0.800821,0.801242,0.801347,0.802504,0.800189,0.799874,0.80061,0.800295,0.799137,0.799242,0.800505
split1_test_score,0.811553,0.812605,0.81271,0.812816,0.812184,0.811763,0.811763,0.811974,0.811974,0.811974,0.810606,0.811343,0.810185,0.809449,0.809028,0.808712,0.809343,0.808817,0.809133,0.80987,0.803872,0.808396,0.808396,0.802189,0.804082,0.803977,0.803977,0.803346,0.804188,0.802083


In [0]:
new_pipeline= search.best_estimator_

In [0]:
trainval = pd.concat([train,val])

In [0]:
trainval

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,basin,region,lga,population,public_meeting,scheme_management,scheme_name,permit,extraction_type_group,management_group,payment,quality_group,quantity,source,source_class,waterpoint_type,status_group,constr_to_inspect
33440,12451,0.0,Government Of Tanzania,1120.0,DWE,36.802143,-3.441487,Pangani,Arusha,Arusha Rural,150,True,VWC,Machumba estate pipe line,True,gravity,user-group,never pay,good,enough,spring,groundwater,other,functional,25.0
31804,40724,0.0,Rwssp,NaN,DWE,32.588081,-3.916705,Lake Tanganyika,Shinyanga,Kahama,0,True,NaN,NaN,True,other,user-group,unknown,milky,dry,machine dbh,groundwater,other,non functional,NaN
50213,48552,0.0,World Bank,330.0,Kalumbwa,37.555691,-6.484591,Wami / Ruvu,Morogoro,Mvomero,1,True,VWC,Borehole,True,submersible,user-group,never pay,good,insufficient,machine dbh,groundwater,communal standpipe,non functional,4.0
37358,59714,500.0,Norad,938.0,RWE,29.674607,-4.781381,Lake Tanganyika,Kigoma,Kigoma Rural,120,True,VWC,Mkongoro One,True,gravity,user-group,pay monthly,good,insufficient,river,surface,communal standpipe multiple,functional needs repair,28.0
50476,44779,100.0,World Bank,1320.0,Government,36.529419,-5.681505,Wami / Ruvu,Manyara,Kiteto,500,True,Water Board,Water AID,True,submersible,user-group,pay per bucket,good,insufficient,machine dbh,groundwater,communal standpipe multiple,non functional,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40650,34313,0.0,World Vision,NaN,World vision,32.166262,-3.632291,Lake Tanganyika,Shinyanga,Bukombe,0,NaN,VWC,NaN,True,other,user-group,unknown,good,dry,shallow well,groundwater,other,non functional,NaN
15194,14379,0.0,NaN,NaN,NaN,33.908171,-9.291460,Lake Nyasa,Mbeya,Rungwe,0,True,Parastatal,K,NaN,gravity,parastatal,unknown,good,enough,spring,groundwater,communal standpipe,non functional,NaN
14819,58553,0.0,Netherlands,NaN,DWE,33.745930,-3.061495,Lake Victoria,Shinyanga,Maswa,0,True,WUG,NaN,False,nira/tanira,user-group,pay when scheme fails,good,enough,shallow well,groundwater,hand pump,functional,NaN
17938,2101,0.0,Oxfarm,NaN,DWE,31.156742,-1.708934,Lake Victoria,Kagera,Karagwe,0,True,VWC,Kishuro water sup,True,gravity,user-group,never pay,good,enough,spring,groundwater,communal standpipe,functional,NaN


In [0]:
X_trainval = trainval[features]
y_trainval = trainval[target]

In [0]:
new_pipeline.fit(X_trainval,y_trainval)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['lga', 'scheme_name', 'funder',
                                      'installer'],
                                drop_invariant=False, handle_missing='value',
                                handle_unknown='value',
                                mapping=[{'col': 'lga', 'data_type': dtype('O'),
                                          'mapping': Arusha Rural      1
Kahama            2
Mvomero           3
Kigoma Rural      4
Kiteto            5
               ... 
Liwale          122
Kigoma Urban    123
Lindi Urban     124
Nyamagana       125
NaN              -2
Length: 126, dtype:...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
      

In [0]:
y_pred = new_pipeline.predict(X_test)

In [0]:
sample_submission['status_group'] = y_pred

In [0]:
sample_submission.to_csv('submission.csv',index=False)

In [0]:
search2 = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter= 30,
    cv=5,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    random_state=13,
    n_jobs=-1
)

In [0]:
search2.fit(X_trainval,y_trainval)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   38.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 10.8min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=['lga',
                                                                   'scheme_name',
                                                                   'funder',
                                                                   'installer'],
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('one

In [0]:
pipeline2 = search2.best_estimator_

In [0]:
y_pred2 = pipeline2.predict(X_test)

In [0]:
sample_submission['status_group'] = y_pred2

In [0]:
sample_submission.to_csv('submission.csv', index=False)